In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# download GlenCove excel data
df = pd.read_excel("GlenCove.xlsx")

In [3]:
# display first ten rows of data
df.head(n=10)

,Address,Fair Market Value($000),Property Size (acres),Age,House Size (square feet),Rooms,Baths,Garage
0,9 Sycamore Road,522.9,0.2297,56,2448,7,3.5,2
1,21 Jefferson St,425.0,0.2192,61,1942,7,2.5,1
2,38 Hitching Post Lane,539.2,0.1630,39,2073,5,3.0,2
3,4 Poppy Lane,628.2,0.4608,28,2707,8,2.5,1
4,5 Daniel Drive,490.4,0.2549,56,2042,7,1.5,1
5,15 Francis Terrace,487.7,0.2290,98,2089,7,2.0,0
6,23 Guilfoy Street,370.3,0.1808,58,1433,7,2.0,0
7,17 Carlyle Drive,777.9,0.5015,17,2991,9,2.5,1
8,8 Craft Avenue,347.1,0.2229,62,1008,5,1.0,0
9,22 Beechwood Ct.,756.8,0.1300,25,3202,8,2.5,2


In [4]:
# renames market value, property size, and house size columns
df.columns = ['Address', 'market_value', 'property_size', 'Age', 'house_size', 'Rooms', 'Baths','Garage']

In [5]:
# creates new dataframe without age column
newdf = df.drop(columns = 'Age')

# sets Garage to binary of 0 for no garage, and 1 for garage
# Garage data contains only values 0, 1, and 2 - only need to replace 2 with 1
newdf["Garage"] = newdf.Garage.replace({2:1})

In [6]:
# print summary stats for remaining columns
print(newdf.describe())

       market_value  property_size   house_size      Rooms      Baths  \
count     30.000000      30.000000    30.000000  30.000000  30.000000   
mean     474.910000       0.275960  1978.833333   6.833333   2.000000   
std      144.312669       0.236445   550.875234   1.487496   0.694808   
min      310.200000       0.085200  1008.000000   5.000000   1.000000   
25%      374.675000       0.152650  1622.250000   6.000000   1.625000   
50%      431.200000       0.204050  1992.000000   7.000000   2.000000   
75%      535.125000       0.270350  2205.500000   7.750000   2.500000   
max      889.000000       1.310000  3202.000000  11.000000   3.500000   

          Garage  
count  30.000000  
mean    0.566667  
std     0.504007  
min     0.000000  
25%     0.000000  
50%     1.000000  
75%     1.000000  
max     1.000000  


In [7]:
# print correlation for prop size, house size, rooms and baths
print(newdf[['property_size','house_size','Rooms','Baths']].corr())

               property_size  house_size     Rooms     Baths
property_size       1.000000    0.211664  0.052963  0.053408
house_size          0.211664    1.000000  0.399573  0.521314
Rooms               0.052963    0.399573  1.000000  0.133457
Baths               0.053408    0.521314  0.133457  1.000000


All of the correlations are positive in nature, with varying levels of correlation
1. property_size and house_size = 0.211664
2. property_size and Rooms = 0.052963
3. property_size and Baths = 0.053408
4. house_size and Rooms = 0.399573
5. house_size and Baths = 0.521314
6. Rooms and Baths = 0.133457

In [8]:
# create linear regression, then create and print summary
results = sm.formula.ols('market_value ~ property_size + house_size + Rooms + Baths + Garage', newdf).fit()
results_sum = results.summary()
print(results_sum)

                            OLS Regression Results                            
Dep. Variable:           market_value   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     21.09
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           4.52e-08
Time:                        22:03:20   Log-Likelihood:                -165.94
No. Observations:                  30   AIC:                             343.9
Df Residuals:                      24   BIC:                             352.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        78.0523     67.779      1.152

In [9]:
# creates and prints anova table
aov_table = sm.stats.anova_lm(results, typ = 2)
print(aov_table)

                      sum_sq    df          F    PR(>F)
property_size  185956.980096   1.0  39.850226  0.000002
house_size      45335.829171   1.0   9.715382  0.004693
Rooms              27.229339   1.0   0.005835  0.939743
Baths           13203.326340   1.0   2.829448  0.105518
Garage           3736.437085   1.0   0.800711  0.379768
Residual       111993.529961  24.0        NaN       NaN


This model actually performs very well. The R-squared value is relatively high in the range at 0.815, and the probability is 4.52e-08. The probability alone is well below the 0.05 threshold to reject a null hypothesis. Most of the value in this model is coming from property_size and house_size, not much clarity would be lost if Rooms, Baths and Garage were excluded from the regression.

The regression equation would be:

market_value = 78.0523 + 359.3104 * property_size + 0.1025 * house_size + 0.7385 * Rooms + 36.8445 * Baths + 28.3802 * Garage

The fair market value ($000) of a house that is 2000 sqft, on 0.30 acres, with 6 rooms, 2 bathrooms, and has a garage would be 497.34562